### Anomaly detection using ViT & U-Net Model (Stage 2)

In [ ]:
import os #to joint the path
from glob import glob #used to extract images and masks path
from sklearn.model_selection import train_test_split #to split the dataset to train and validation
from tqdm import tqdm #The progress bar
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
def create_dir(path):
  """ Create a directory """
  if not os.path.exists(path):
    os.makedirs(path)

### Prepar train and valid dataset

In [ ]:
create_dir('Data')
create_dir('Data/Dataset')
create_dir('Data/TrainValid')

In [ ]:
# Paths
dataset_path = '/content/Data/Dataset/ped1'

In [ ]:
# Copy Data folder from drive
!cp -r '/content/drive/MyDrive/Colab Notebooks/ADViTUNet/Data/Preprocess/' '/content/Data/'
!cp -r '/content/drive/MyDrive/Colab Notebooks/ADViTUNet/Dataset/ped1/' '/content/Data/Dataset/'

In [ ]:
!cp -r '/content/Data/Preprocess/FramesComb/' '/content/Data/Dataset/ped1'

In [ ]:
from sklearn.utils.validation import sp
def load_data(path,split=0.2):
  """ Load frames and masks """
  frames = sorted(glob(f"{path}/FramesComb/*.jpg"))
  masks = sorted(glob(f"{path}/Frames_GT/*.bmp"))
  print("Number of frames : ",len(frames))
  print("Number of masks : ", len(masks))

  """ Split the data """
  split_size = int(len(frames) * split)
  train_x, valid_x = train_test_split(frames, test_size=split_size, random_state=42)
  train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

  return (train_x, train_y), (valid_x, valid_y)

In [ ]:
""" Load tha dataset """
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path,split=0.2)

' Load tha dataset '

In [ ]:
create_dir("/content/Data/TrainValid/train/frames/")
create_dir("/content/Data/TrainValid/train/masks/")
create_dir("/content/Data/TrainValid/valid/frames/")
create_dir("/content/Data/TrainValid/valid/masks/")

In [ ]:
# Paths
TrainPath = '/content/Data/TrainValid/train/'
ValidPath = '/content/Data/TrainValid/valid/'

In [ ]:
from albumentations import HorizontalFlip, VerticalFlip, Rotate #Data augmentation

In [ ]:
def augment_data(images, masks, save_path, augment=False):
  """ Performing data augmentation """
  H = 512
  W = 512

  for idx, (x,y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
    """Extracting the dir name and image name"""
    dir_name=x.split("/")[-3]
    name=dir_name + "_" + x.split("/")[-1].split(".")[0]

    """Read the image and name"""
    x=cv2.imread(x, cv2.IMREAD_COLOR)
    y=cv2.imread(y, cv2.IMREAD_COLOR)

    if augment== True:
      aug = HorizontalFlip(p=1.0) #p:probability of applying this data augmentation
      augmented = aug(image=x, mask=y)
      x1=augmented["image"]
      y1=augmented["mask"]

      aug = VerticalFlip(p=1)
      augmented = aug(image=x, mask=y)
      x2=augmented["image"]
      y2=augmented["mask"]

      aug = Rotate(limit=45, p=1.0)
      augmented = aug(image=x, mask=y)
      x3=augmented["image"]
      y3=augmented["mask"]

      X=[x, x1, x2, x3]
      Y=[y, y1, y2, y3]

    else:
      X=[x]
      Y=[y]

    idx = 0
    for i, m in zip(X, Y):
      i=cv2.resize(i,(W,H))
      m=cv2.resize(m, (W,H))
      m=m/255.0
      m=(m > 0.5)*255

      if len(X) == 1:
        tmp_image_name = f"{name}.jpg"
        tmp_mask_name = f"{name}.jpg"
      else:
        tmp_image_name = f"{name}_{idx}.jpg"
        tmp_mask_name = f"{name}_{idx}.jpg"

      image_path = os.path.join(save_path,"frames/", tmp_image_name)
      mask_path = os.path.join(save_path,"masks/", tmp_mask_name)

      cv2.imwrite(image_path, i)
      cv2.imwrite(mask_path, m)

      idx += 1

    # break


In [1]:
# augment_data(train_x, train_y,TrainPath, augment=True)
augment_data(train_x, train_y, TrainPath, augment=False)
augment_data(valid_x, valid_y, ValidPath, augment=False) #we dont apply data augmentation for validation data

##Model U-Net

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
#Model:is a model class that take an input image and output mask

In [ ]:
#Create convolution block
def conv_block(input, num_filters):
  x = Conv2D(num_filters, 3, padding="same")(input)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  x = Conv2D(num_filters, 3, padding="same")(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)

  return x

In [ ]:
#Encoder block
def encoder_block(input, num_filter):
  x = conv_block(input, num_filter) #x is skip connection
  p = MaxPool2D((2, 2))(x) # p is a feature for next block
  return x, p

In [ ]:
def decoder_block(input, skip_features, num_filters):
  # print(input.shape)
  x = Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(input)
  # print(x.shape)
  x = Concatenate()([x, skip_features])
  # print(x.shape)
  x = conv_block(x, num_filters)
  return x

In [ ]:
def build_unet(input_shape):
  inputs = Input(input_shape)
#The encoder part
  s1, p1 = encoder_block(inputs, 64) #64:number of filters
  s2, p2 = encoder_block(p1, 128)
  s3, p3 = encoder_block(p2, 256)
  s4, p4 = encoder_block(p3, 512)
#the resolution decreases and the number of filter increases, thats the rule
#Bottle neck part (its conv block, not Pool there)
  b1 = conv_block(p4, 1024)

#The decoder part
  # d1 = decoder_block(b1,512) V1
  d1 = decoder_block(b1,s4, 512) #V2:add skip connection

  # print(p1.shape, p2.shape, p3.shape, p4.shape)
  # print(s1.shape, s2.shape, s3.shape, s4.shape)
  #la last shape(s4) is the same shape required
  d2 = decoder_block(d1,s3, 256)
  d3 = decoder_block(d2,s2, 128)
  d4 = decoder_block(d3,s1, 64)
#in the decoder part the resolution increases and the number of filter decreases

  outputs = Conv2D(1,1, padding="same", activation="sigmoid")(d4)
  #1 : number of filters
  #1 : number of colon size
  #one bcz we have one classe only (mask)

  model = Model(inputs, outputs, name="U-Net")
  return model


In [ ]:
#Main
input_shape = (512,512,3)
model = build_unet(input_shape)
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 512, 512, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 512, 512, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 512, 512, 64)         0         ['batch_normalization[0][0

In [ ]:
##Trainig dataset
from os.path import ismount
import os
# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

In [ ]:
H = 512
W = 512
def shuffling(x,y):
  x,y = shuffle(x,y, random_state=42)
  return x,y

def load_new_data(path):
  x = sorted(glob(os.path.join(path,"frames","*.jpg")))
  y = sorted(glob(os.path.join(path,"masks","*.jpg")))
  return x,y

def read_image(path):
  path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_COLOR)
  # x = cv2.resize(x, (W, H))
  x = x/255.0
  x = x.astype(np.float32)
  return x

def read_mask(path):
  path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  # x = cv2.resize(x, (W, H))
  x = x/255.0
  x = x > 0.5
  x = x.astype(np.float32)
  x = np.expand_dims(x, axis=-1)
  return x

#Build dataset piplin
def tf_parse(x,y):
  def _parse(x,y):
    x = read_image(x)
    y = read_mask(y)
    return x,y

  x,y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
  x.set_shape([H, W, 3])
  y.set_shape([H, W, 1])
  return x,y

def tf_dataset(x,y, batch=8):
  #x,y : list of images and masks path
  dataset = tf.data.Dataset.from_tensor_slices((x,y))
  dataset = dataset.map(tf_parse)
  dataset = dataset.batch(batch)
  dataset = dataset.prefetch(10)
  return dataset


In [ ]:
#Main
"""Seeding"""
np.random.seed(42)
tf.random.set_seed(42)

"""Directory for storing files"""
create_dir("files")


""" Hyperparameters """
batch_size = 2
lr = 1e-4
num_epochs = 20
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files","data.csv")

"""Dataset"""
train_x, train_y = load_new_data(TrainPath)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_new_data(ValidPath)

In [ ]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

##Metrics

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import  backend as k

In [ ]:
def iou(y_true, y_pred):
  def f(y_true, y_pred):
    intersection = (y_true * y_pred).sum()
    union = y_true.sum() + y_pred.sum() - intersection
    x = (intersection + 1e-15) / (union + 1e-15)
    x = x.astype(np.float32)
    return x
  return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15

def dice_coef(y_true, y_pred):
  y_true = tf.keras.layers.Flatten()(y_true)
  y_pred = tf.keras.layers.Flatten()(y_pred)
  intersection = tf.reduce_sum(y_true * y_pred)
  return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)+smooth)

def dice_loss(y_true, y_pred):
  return 1.0 - dice_coef(y_true, y_pred)

##training the model

In [2]:
H = 512
W = 512
input_shape = (H,W,3)
model = build_unet(input_shape)
metrics = [dice_coef, iou, Recall(), Precision()]
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

#This callback use will training
callbacks = [
             ModelCheckpoint(model_path, verbose=1,save_best_only=True),
             ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=10, min_lr=1e-7,verbose=1),
             CSVLogger(csv_path),
             TensorBoard(),
             EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False)
             #if we achieve 50 epoches and the val_loss doesnt dercease stop training
             ]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    # steps_per_epoch=train_steps,
    # validation_steps=valid_steps,
    callbacks=callbacks,
    shuffle=False
)


In [ ]:
!cp -r '/content/files' '/content/drive/MyDrive/Colab Notebooks/ADViTUNet/Models/UModel_1/'
!cp -r '/content/logs' '/content/drive/MyDrive/Colab Notebooks/ADViTUNet/Models/UModel_1/'

##Evaluation

In [ ]:
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score,recall_score

In [ ]:
"""Seeding"""
np.random.seed(42)
tf.random.set_seed(42)

"""Directory for storing results"""
create_dir("Results")

"""Loading model"""
with CustomObjectScope({'iou':iou, 'dice_coef':dice_coef, 'dice_loss':dice_loss}):
  model = tf.keras.models.load_model("files/model.h5")
  model.summary()


Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 512, 512, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_18 (Ba  (None, 512, 512, 64)         256       ['conv2d_19[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_18 (Activation)  (None, 512, 512, 64)         0         ['batch_normalization_18[0

In [ ]:
"""Load the dataset"""
test_x = sorted(glob(os.path.join("Data", "TrainValid", "valid", "frames","*")))
test_y = sorted(glob(os.path.join("Data", "TrainValid", "valid", "masks","*")))

In [ ]:
#Function
def save_results(image, mask, y_pred, save_image_path):
  ## i - m - y
  line = np.ones((H, 10, 3)) * 128

  """ Mask """
  mask = np.expand_dims(mask, axis=-1) ##(512,512,1)
  mask = np.concatenate([mask,mask,mask], axis=-1) #(512,512,3)

  """ Predicted Mask """
  y_pred = np.expand_dims(y_pred, axis=-1) ##(512,512,1)
  y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) #(512,512,3)
  y_pred = y_pred * 255

  cat_images = np.concatenate([image, line, mask, line, y_pred], axis=1)
  cv2.imwrite(save_image_path, cat_images)


In [3]:
import pandas as pd
""" Evaluation and Prediction """
SCORE = []
for x,y in tqdm(zip(test_x,test_y), total=len(test_x)):
  """Extract the name"""
  name = x.split("/")[-1].split(".")[0]
  print(name)

  """Reading the image"""
  image = cv2.imread(x, cv2.IMREAD_COLOR)
  x = image/255.0
  x = np.expand_dims(x, axis=0) #bcz the data is a batch format

  """Reading the mask"""
  mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
  y = mask/255.0
  y = y > 0.5
  y = y.astype(np.int32)

  """Prediction"""
  y_pred = model.predict(x)[0]
  y_pred = np.squeeze(y_pred, axis=-1)
  # print(y_pred.shape)
  y_pred = y_pred > 0.5
  y_pred = y_pred.astype(np.int32)

  """Saving the prediction"""
  save_image_path=f"Results/{name}.png"
  save_results(image, mask, y_pred, save_image_path)

  """Flatten the array"""
  y = y.flatten()
  y_pred = y_pred.flatten()

  """Calculating the metrics values"""
  acc_value = accuracy_score(y, y_pred)
  f1_value = f1_score(y, y_pred, labels=[0,1], average="binary", zero_division=1)
  jac_value = jaccard_score(y, y_pred, labels=[0,1], average="binary", zero_division=1)
  recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary", zero_division=1)
  precision_value = precision_score(y,y_pred, labels=[0,1], average="binary", zero_division=1)
  SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])


"""Metrics value"""
score = [s[1:]for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy:{score[0]:0.5f}")
print(f"F1:{score[1]:0.5f}")
print(f"Jaccard:{score[2]:0.5f}")
print(f"Recall:{score[3]:0.5f}")
print(f"Precision:{score[4]:0.5f}")


df = pd.DataFrame(SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
df.to_csv("files/score.csv")


In [ ]:
#Save results to Colab
!cp -r '/content/Results' '/content/drive/MyDrive/Colab Notebooks/ADViTUNet/UResults'